In [2]:
import tensorflow as tf
from model_trainer import *
import matplotlib.pyplot as plt
from tensorflow.keras.mixed_precision import experimental as mixed_precision
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 2070 SUPER, compute capability 7.5


In [3]:
import cv2
import numpy as np
from pathlib import Path

In [4]:
import backbone_models
import specific_models

In [5]:
import skimage

In [6]:
vid_path = 'data/test/08_25_05_30.h264'
model_path = 'savedmodels/hr538_316/100'
bb_model = backbone_models.hr_5_3_8
sp_model = {
    'nose' : specific_models.conv3_16,
    'tail' : specific_models.conv3_16,
}

In [7]:
testmodel = ChaserModel(tf.keras.Input((240,320,3)),bb_model,sp_model)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 240, 320, 3) 0                                            
__________________________________________________________________________________________________
HR_0 (HighResolutionModule)     [(None, 240, 320, 8) 3760        input_1[0][0]                    
__________________________________________________________________________________________________
HR_1 (HighResolutionModule)     [(None, 240, 320, 8) 19336       HR_0[0][0]                       
__________________________________________________________________________________________________
HR_2 (HighResolutionModule)     [(None, 240, 320, 8) 93088       HR_1[0][0]                       
                                                                 HR_1[0][1]            

In [8]:
testmodel.load_weights(model_path)

In [9]:
original_size = (640,480)
original_hw = (original_size[1],original_size[0])
model_size = (320,240)
model_hw = (model_size[1],model_size[0])
batch_size = 196

In [13]:
cap = cv2.VideoCapture(vid_path)
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('result.mp4',fourcc,30,original_size)
frames = []
original_frames=[]
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        small_frame = cv2.resize(frame, dsize=model_size)[...,2::-1]
        frames.append(small_frame)
        original_frames.append(frame)
    else:
        break
print('loaded')
batch_num = len(frames) // batch_size
count = 0
for i in range(batch_num):
    print(f'{i+1}th/{batch_num} batch')
    output = testmodel(np.array(frames[i*batch_size:(i+1)*batch_size]))
    nose_hms, tail_hms = output['nose'].numpy(), output['tail'].numpy()
    nose_poses = np.unravel_index(nose_hms.reshape((nose_hms.shape[0],-1)).argmax(axis=1),nose_hms.shape[1:])
    nose_poses = np.swapaxes(nose_poses,0,1)
    tail_poses = np.unravel_index(tail_hms.reshape((tail_hms.shape[0],-1)).argmax(axis=1),tail_hms.shape[1:])
    tail_poses = np.swapaxes(tail_poses,0,1)
    for nose_pos, tail_pos in zip(nose_poses, tail_poses):
        new_nose = np.multiply(nose_pos,np.divide(original_hw,model_hw)).astype(np.int)
        new_tail = np.multiply(tail_pos,np.divide(original_hw,model_hw)).astype(np.int)
        nose_rr, nose_cc = skimage.draw.disk(new_nose,10, shape=original_hw)
        tail_rr, tail_cc = skimage.draw.disk(new_tail,10, shape=original_hw)
        frame = original_frames[count]
        frame[nose_rr,nose_cc] = [0,255,0]
        frame[tail_rr,tail_cc] = [255,0,0]
        writer.write(frame)
        count += 1

cap.release()
writer.release()
print('done')

loaded
1th/29 batch
2th/29 batch
3th/29 batch
4th/29 batch
5th/29 batch
6th/29 batch
7th/29 batch
8th/29 batch
9th/29 batch
10th/29 batch
11th/29 batch
12th/29 batch
13th/29 batch
14th/29 batch
15th/29 batch
16th/29 batch
17th/29 batch
18th/29 batch
19th/29 batch
20th/29 batch
21th/29 batch
22th/29 batch
23th/29 batch
24th/29 batch
25th/29 batch
26th/29 batch
27th/29 batch
28th/29 batch
29th/29 batch
done
